In [1]:
# Human specific
import inspect

# This prompt is ignored if the environment_name already corresponds to an existing environment.
specific = inspect.cleandoc("""\
    You are an expert at scheduling meetings. 
    You are given a few constraints on the existing schedule of each participant, the meeting duration, and possibly some preferences on the meeting time. 
    Note there exists a solution that works with existing schedule of every participant. 
    TASK: 
    You need to schedule a meeting for Michelle, Steven and Jerry for one hour between the work hours of 9:00 to 17:00 on Monday. 
    
    Here are the existing schedules for everyone during the day: 
    Michelle has meetings on Monday during 11:00 to 12:00; 
    Steven has blocked their calendar on Monday during 9:00 to 9:30, 11:30 to 12:00, 13:30 to 14:00, 15:30 to 16:00; 
    Jerry has blocked their calendar on Monday during 9:00 to 9:30, 10:00 to 11:00, 11:30 to 12:30, 13:00 to 14:30, 15:30 to 16:00, 16:30 to 17:00; 
    
    Find a time that works for everyone's schedule and constraints.
    """)
environment_name = "CalendarScheduling"
format = "json"

In [2]:
# Generate the json representation of the environment
from pathlib import Path

from src.llm_plan.planner import Planner
from src.llm_plan.llm import ChatGPT
from src.llm_plan.config import ENVIRONMENTS_JSON_PATH

model = ChatGPT("gpt-5-mini")  # This is one of the many models you can use now
planner = Planner()

plan_path = Path(f"{environment_name}.{format}")
full_path = ENVIRONMENTS_JSON_PATH / plan_path

# Skip if the plan already exists
if not full_path.exists():
    planner.generate_representation(model, specific, environment_name, format=format)
else:
    print(f"{full_path} already exists. Skipping generation.")
    print("[Warning]: The prompt `specific` will be ignored!")

environments/static/CalendarScheduling.json already exists. Skipping generation.
[Warning]: The prompt `specific` will be ignored!


In [3]:
# Generate the environment plan
from src.llm_plan.environment import Environment

env = Environment(f"./environments/static/{environment_name}.json")
print("Plan:\n", env.plan)

Plan:
 [['michelle.pddl', 'steven.pddl', 'jerry.pddl'], ['orchestrator.pddl']]


In [4]:
# Plan
model = ChatGPT("gpt-4o")  # switch to a less powerful model for planning
responses = planner.plan(model, env)

----- michelle->pddl -----

----- System Prompt -----
You are an expert with PDDL problems (Planning Domain Definition Language). You always provide a PDDL domain and a PDDL problem file to solve the task. You always enclose the pddl domain between <domain></domain> tags, and the pddl problem between <problem></problem> tags.

----- Prompt -----
Your name is michelle. You are in an environment with the following public information: ['Work hours are 09:00 to 17:00 on Monday.', 'The meeting duration required is 60 minutes.', 'Time granularity is 30 minutes.', 'There exists at least one solution that fits all participant schedules.'] You have the following knowledge: ['Monday 11:00-12:00'] This is your goal: Schedule a one-hour meeting on Monday between 09:00 and 17:00 that works for all participants. Think step by step and provide a PDDL domain and a PDDL problem file that represent your availability constraints and the meeting requirement. If you miss some information, do not make assum

In [5]:
# Extract the pddl problem and domain from the LLM response
from src.llm_plan.parser import PDDLParser

pddl_parser = PDDLParser()

# TODO: fix the vocabulary entry, it shouldn't be "pddl_scheduler" etc.
final_plan = responses["pddl_orchestrator"]
domain, problem = pddl_parser.parse(final_plan, from_file=False)

In [6]:
# Obtain the plan with the solver
import subprocess

from pathlib import Path

from src.llm_plan.config import SOLVER_BINARY, SOLVER_ARGS

BASE_FOLDER = Path(f"./tmp/{env.name}")
BASE_FOLDER.mkdir(parents=True, exist_ok=True)

with open(BASE_FOLDER / "problem.pddl", "w") as f:
    f.write(problem)

with open(BASE_FOLDER / "domain.pddl", "w") as f:
    f.write(domain)
    
# Launch the solver
command = [
    SOLVER_BINARY,
    *SOLVER_ARGS,
    BASE_FOLDER / "sas_plan",
    BASE_FOLDER / "domain.pddl",
    BASE_FOLDER / "problem.pddl",
]

with open(BASE_FOLDER / "logs.txt", "w") as logfile:
    subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)

In [7]:
# Validate the plan with uVAL
import subprocess

from src.llm_plan.config import UNIVERSAL_VALIDATOR_BIN

command = f"{UNIVERSAL_VALIDATOR_BIN} -cv \
{BASE_FOLDER / 'domain.pddl'} \
{BASE_FOLDER / 'problem.pddl'} \
{BASE_FOLDER / 'sas_plan'}"

out = subprocess.run(command, shell=True, capture_output=True, text=True)

# This part won't be printed at test time
if out.stderr:
    pddl_error = out.stderr
    print(
        f"The validation found a problem with the plan: {out.stderr}"
    )
else:
    pddl_error = "No error found."
    print("The plan is valid.")
    print("[stdout]", out.stdout)

The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::ExpectedToken: ( expected



In [8]:
# Let's try to refine the plan syntax
from src.llm_plan.hypervisor import (
    HypervisorDeepThinkPDDL,
    HypervisorSyntaxPDDL,
    HypervisorFastDownwardAdapter
    )

domain_name = "refined_domain_{i}.pddl"
problem_name = "refined_problem_{i}.pddl"
sas_plan_name = "refined_sas_plan_{i}"
logs_name = "refined_logs_{i}.txt"

num_attempts = 10
last_valid_plan = None

for i in range(num_attempts):
    
    # --- General validator of PDDL plans ---
    prompt_args_deepthink = {
        "specification": env.config_data,
        "pddl_domain": domain,
        "pddl_problem": problem,
    }

    deep_think = HypervisorDeepThinkPDDL(llm=model, prompt_args=prompt_args_deepthink)

    r = deep_think.run()
    domain, problem = pddl_parser.parse(r, from_file=False)
    
    # --- Fast Downward adapter ---
    prompt_args_ff_adapter = {
        "specification": env.config_data,
        "pddl_domain": domain,
        "pddl_problem": problem,
    }
    
    hypervisor_fd = HypervisorFastDownwardAdapter(llm=model, prompt_args=prompt_args_ff_adapter)
    
    r = hypervisor_fd.run()
    domain, problem = pddl_parser.parse(r, from_file=False)
    
    
    # Obtain the new plan with the solver
    with open(BASE_FOLDER / problem_name.format(i=i), "w") as f:
        f.write(problem)

    with open(BASE_FOLDER / domain_name.format(i=i), "w") as f:
        f.write(domain)
        
    # --- Launch the solver ---
    command = [
        SOLVER_BINARY,
        *SOLVER_ARGS,
        BASE_FOLDER / sas_plan_name.format(i=i),
        BASE_FOLDER / domain_name.format(i=i),
        BASE_FOLDER / problem_name.format(i=i),
    ]

    with open(BASE_FOLDER / logs_name.format(i=i), "w") as logfile:
        subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)
        
    command = f"{UNIVERSAL_VALIDATOR_BIN} -cv \
    {BASE_FOLDER / domain_name.format(i=i)} \
    {BASE_FOLDER / problem_name.format(i=i)} \
    {BASE_FOLDER / sas_plan_name.format(i=i)}"

    out = subprocess.run(command, shell=True, capture_output=True, text=True)
    
    # Store the last valid plan
    if (BASE_FOLDER / sas_plan_name.format(i=i)).exists():
        last_valid_plan = i

    if out.stderr:
        pddl_error = out.stderr
        print(
            f"The validation found a problem with the plan: {out.stderr}"
        )
    else:
        pddl_error = "No error was found."
        print("The plan is valid.")
        print("[stdout]", out.stdout)
    
    # --- Syntax validator ---
    print(f"Validation attempt {i+1}/{num_attempts}")
    
    with open(BASE_FOLDER / logs_name.format(i=i), "r") as f:
        pddl_logs = f.read()
        
    prompt_args_syntax_pddl = {
        "specification": env.config_data,
        "pddl_domain": domain,
        "pddl_problem": problem,
        "syntax_errors": pddl_error,
        "pddl_logs": pddl_logs
    }

    hypervisor_pddl = HypervisorSyntaxPDDL(llm=model, prompt_args=prompt_args_syntax_pddl)

    r = hypervisor_pddl.run()
    domain, problem = pddl_parser.parse(r, from_file=False)

The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::ExpectedToken: ( expected

Validation attempt 1/10
The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::UnknownToken: MICHELLE does not name a known token

Validation attempt 2/10
The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::UnknownToken: MICHELLE does not name a known token

Validation attempt 3/10
The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::UnknownToken: MICHELLE does not name a known token

Validation attempt 4/10
The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::UnknownToken: MICHELLE does not name a known token

Validation attempt 5/10
The validation found a problem with the plan: libc++abi: ter

In [16]:
# Final plan run and validation
assert last_valid_plan is not None, "No valid plan was found with refinements."

with open(BASE_FOLDER / problem_name.format(i=last_valid_plan), "w") as f:
    f.write(problem)

with open(BASE_FOLDER / domain_name.format(i=last_valid_plan), "w") as f:
    f.write(domain)
    
# Launch the solver
command = [
    SOLVER_BINARY,
    *SOLVER_ARGS,
    BASE_FOLDER / sas_plan_name.format(i=last_valid_plan),
    BASE_FOLDER / domain_name.format(i=last_valid_plan),
    BASE_FOLDER / problem_name.format(i=last_valid_plan),
]

with open(BASE_FOLDER / logs_name.format(i=i), "w") as logfile:
    subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)
    
command = f"{UNIVERSAL_VALIDATOR_BIN} -cv \
{BASE_FOLDER / domain_name.format(i=last_valid_plan)} \
{BASE_FOLDER / problem_name.format(i=last_valid_plan)} \
{BASE_FOLDER / sas_plan_name.format(i=last_valid_plan)}"

out = subprocess.run(command, shell=True, capture_output=True, text=True)

# This part won't be printed at test time
if out.stderr:
    pddl_error = out.stderr
    print(
        f"The validation found a problem with the plan: {out.stderr}"
    )
else:
    pddl_error = "No error found."
    print("The plan is valid.")
    print("[stdout]", out.stdout)

The validation found a problem with the plan: libc++abi: terminating due to uncaught exception of type parser::pddl::UnknownToken: MICHELLE does not name a known token



In [17]:
# Report the natural language plan
from src.llm_plan.hypervisor import HypervisorNaturalLanguage

with open(BASE_FOLDER / problem_name.format(i=last_valid_plan), "r") as f:
    problem = f.read()

with open(BASE_FOLDER / domain_name.format(i=last_valid_plan), "r") as f:
    domain = f.read()
    
with open(BASE_FOLDER / sas_plan_name.format(i=last_valid_plan), "r") as f:
    plan = f.read()
    
prompt_args = {
    "specification": env.config_data,
    "pddl_domain": domain,
    "pddl_problem": problem,
    "pddl_plan": plan,
}

hypervisor_to_nl = HypervisorNaturalLanguage(llm=model, prompt_args=prompt_args)

natural_plan = hypervisor_to_nl.run()

with open(BASE_FOLDER / "refined_nl_plan.txt", "w") as f:
    f.write(natural_plan)